In [1]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
from os import path
import os,binascii
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from sklearn.metrics import mean_squared_error, f1_score
from keras.models import load_model
from sklearn.model_selection import ParameterGrid
%matplotlib inline
from argparse import Namespace

Using TensorFlow backend.


In [2]:
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = [32]
FLAGS.seq_window_lengths = [6]
FLAGS.smi_window_lengths = [4]
FLAGS.batch_size = 256
FLAGS.num_epoch = 100
FLAGS.max_seq_len = [1000]
FLAGS.max_smi_len = 95
FLAGS.dataset_path= '../data/davis/'
FLAGS.include_davis = [False, True]
FLAGS.problem_type = 1
FLAGS.bns = [False]
FLAGS.dropouts = [0.5]
FLAGS.embedding_dropouts = [0.2, 0.5]
FLAGS.charseqset_size = CHARPROTLEN 
FLAGS.charsmiset_size = CHARISOSMILEN 
FLAGS.results_pickle = ['results_kd.pkl']
FLAGS.train_on_KD = [False, True]
FLAGS_dict = vars(FLAGS)

for i, val in FLAGS_dict.items():
    if type(FLAGS_dict[i]) != list:
        FLAGS_dict[i] = [val]

In [3]:
#proteins = json.load(open(FLAGS.dataset_path+"proteins.txt"), object_pairs_hook=OrderedDict)
#proteins = [i[1] for i in proteins.items()]
#dtc_train = pd.read_csv(dtc_data_file)
#with open('proteins.csv', 'w') as handler:
#    for i, seq in enumerate(list(dtc_train['fasta'].unique()) + proteins):
#        handler.write('>id'+str(i)+'\n')
#        handler.write(seq+'\n')

In [4]:
n_repeats = 1
param_grid = ParameterGrid(FLAGS_dict)

if path.exists(FLAGS.results_pickle[0]):
    with open(FLAGS.results_pickle[0], 'rb') as handle:
        results = pickle.load(handle)
else:
    results = {}

for ind in range(len(param_grid)):
    params = param_grid[ind]
    print(params)
    cur_FLAGS = Namespace(**params)
    all_train_drugs, all_train_prots, all_train_Y = load_data(cur_FLAGS)
    val_inds = get_n_fold_by_drugs(all_train_drugs, 10)
    param_name = str(binascii.b2a_hex(os.urandom(15))).replace("'", '')

    early_stopping_callback = CustomStopper(monitor='val_loss', patience=15, start_epoch=20)
    fold_id, avg_loss, avg_cindex, avg_f1, avg_rmse = 0, 0.0, 0.0, 0.0, 0.0
    for tr_fold, val_fold in val_inds.split():
        if fold_id >= n_repeats:
            break

        model_name='checkpoints/davis_dtc_dta_'+param_name+'fold'+str(fold_id)

        checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        
        XD_train, XT_train, Y_train = all_train_drugs[tr_fold], all_train_prots[tr_fold], all_train_Y[tr_fold]
        XD_val, XT_val, Y_val = all_train_drugs[val_fold], all_train_prots[val_fold], all_train_Y[val_fold]

        gridmodel = build_combined_categorical(cur_FLAGS, cur_FLAGS.num_windows, cur_FLAGS.smi_window_lengths, 
                                       cur_FLAGS.seq_window_lengths, cur_FLAGS.dropouts, apply_bn=cur_FLAGS.bns)

        gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=cur_FLAGS.batch_size, epochs=cur_FLAGS.num_epoch, 
                validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
                   ,callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)
        
        gridmodel = load_model(model_name+'.h5')
        predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val)])[:, 0]
        if cur_FLAGS.train_on_KD:
            predicted_labels = -np.log10((1e-8+predicted_labels)/1e9)
            Y_val = -np.log10(Y_val/1e9)
        
        avg_loss += mean_squared_error(Y_val, predicted_labels)/n_repeats
        avg_cindex += ci(Y_val, predicted_labels)/n_repeats
        avg_f1 += f1_score(Y_val>7, predicted_labels>7)/n_repeats
        avg_rmse += np.sqrt(mean_squared_error(Y_val, predicted_labels))/n_repeats
        fold_id += 1
    
    results[param_name] = {'FLAGS':vars(cur_FLAGS), 'params': params, 'loss': avg_loss, 'cindex': avg_cindex,
                       'rmse':  avg_rmse, 'f1': avg_f1, 'n_repeats': n_repeats}
    with open(FLAGS.results_pickle[0], 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


{'checkpoint_path': '', 'batch_size': 256, 'charseqset_size': 25, 'charsmiset_size': 64, 'num_smi_hidden_CNN': 1, 'problem_type': 1, 'binary_th': 0.0, 'bns': False, 'num_seq_hidden_CNN': 1, 'num_windows': 32, 'max_smi_len': 95, 'results_pickle': 'results_kd.pkl', 'num_FC_layers': 1, 'dataset_path': '../data/davis/', 'seq_window_lengths': 6, 'train_on_KD': False, 'dtc_path': '../../data/dtc_for_deepDTA.csv', 'dropouts': 0.5, 'max_seq_len': 1000, 'smi_window_lengths': 4, 'log_dir': '/tmp1542312606.261054/', 'num_epoch': 100, 'include_davis': False, 'embedding_dropouts': 0.2, 'learning_rate': 0.001}
Reading ../data/davis/ start
Read ../data/davis/ start
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 14408 samples, validate on 1437 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 1.06064, saving model to checkpoints/davis_dtc_dta_beaf5b1662e9d26d052a96a2ec17433fold0.h5


/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'checkpoint_path': '', 'batch_size': 256, 'charseqset_size': 25, 'charsmiset_size': 64, 'num_smi_hidden_CNN': 1, 'problem_type': 1, 'binary_th': 0.0, 'bns': False, 'num_seq_hidden_CNN': 1, 'num_windows': 32, 'max_smi_len': 95, 'results_pickle': 'results_kd.pkl', 'num_FC_layers': 1, 'dataset_path': '../data/davis/', 'seq_window_lengths': 6, 'train_on_KD': True, 'dtc_path': '../../data/dtc_for_deepDTA.csv', 'dropouts': 0.5, 'max_seq_len': 1000, 'smi_window_lengths': 4, 'log_dir': '/tmp1542312606.261054/', 'num_epoch': 100, 'include_davis': False, 'embedding_dropouts': 0.2, 'learning_rate': 0.001}
Reading ../data/davis/ start
Read ../data/davis/ start
Train on 14408 samples, validate on 1437 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 77095397.66736, saving model to checkpoints/davis_dtc_dta_be84a46ff05014d7be837c403cc703bfold0.h5
 - 6s - loss: 5384522957.5391 - cindex_score: 0.5112 - val_loss: 77095397.6674 - val_cindex_score: 0.5312
Epoch 2/100
Epoch 00002: val_loss 

/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'checkpoint_path': '', 'batch_size': 256, 'charseqset_size': 25, 'charsmiset_size': 64, 'num_smi_hidden_CNN': 1, 'problem_type': 1, 'binary_th': 0.0, 'bns': False, 'num_seq_hidden_CNN': 1, 'num_windows': 32, 'max_smi_len': 95, 'results_pickle': 'results_kd.pkl', 'num_FC_layers': 1, 'dataset_path': '../data/davis/', 'seq_window_lengths': 6, 'train_on_KD': False, 'dtc_path': '../../data/dtc_for_deepDTA.csv', 'dropouts': 0.5, 'max_seq_len': 1000, 'smi_window_lengths': 4, 'log_dir': '/tmp1542312606.261054/', 'num_epoch': 100, 'include_davis': True, 'embedding_dropouts': 0.2, 'learning_rate': 0.001}
Reading ../data/davis/ start
Read ../data/davis/ start
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.64189, saving model to checkpoints/davis_dtc_dta_bb8685acc484adb82103aacc7c868e4fold0.h5
 - 16s - loss: 3.4527 - cindex_score: 0.5470 - val_loss: 0.6419 - val_cindex_score: 0.6190
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 13s

/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'checkpoint_path': '', 'batch_size': 256, 'charseqset_size': 25, 'charsmiset_size': 64, 'num_smi_hidden_CNN': 1, 'problem_type': 1, 'binary_th': 0.0, 'bns': False, 'num_seq_hidden_CNN': 1, 'num_windows': 32, 'max_smi_len': 95, 'results_pickle': 'results_kd.pkl', 'num_FC_layers': 1, 'dataset_path': '../data/davis/', 'seq_window_lengths': 6, 'train_on_KD': True, 'dtc_path': '../../data/dtc_for_deepDTA.csv', 'dropouts': 0.5, 'max_seq_len': 1000, 'smi_window_lengths': 4, 'log_dir': '/tmp1542312606.261054/', 'num_epoch': 100, 'include_davis': True, 'embedding_dropouts': 0.2, 'learning_rate': 0.001}
Reading ../data/davis/ start
Read ../data/davis/ start
Train on 41811 samples, validate on 4090 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 17777563.54597, saving model to checkpoints/davis_dtc_dta_b9711fe04a1d681bd79abb7cb9b1c93fold0.h5
 - 16s - loss: 1864572681.6497 - cindex_score: 0.5148 - val_loss: 17777563.5460 - val_cindex_score: 0.5441
Epoch 2/100
Epoch 00002: val_loss 

/home/ibrahim/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'checkpoint_path': '', 'batch_size': 256, 'charseqset_size': 25, 'charsmiset_size': 64, 'num_smi_hidden_CNN': 1, 'problem_type': 1, 'binary_th': 0.0, 'bns': False, 'num_seq_hidden_CNN': 1, 'num_windows': 32, 'max_smi_len': 95, 'results_pickle': 'results_kd.pkl', 'num_FC_layers': 1, 'dataset_path': '../data/davis/', 'seq_window_lengths': 6, 'train_on_KD': False, 'dtc_path': '../../data/dtc_for_deepDTA.csv', 'dropouts': 0.5, 'max_seq_len': 1000, 'smi_window_lengths': 4, 'log_dir': '/tmp1542312606.261054/', 'num_epoch': 100, 'include_davis': False, 'embedding_dropouts': 0.5, 'learning_rate': 0.001}
Reading ../data/davis/ start
Read ../data/davis/ start
Train on 14408 samples, validate on 1437 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 1.87180, saving model to checkpoints/davis_dtc_dta_b535a1de9481241048da88849285f7efold0.h5
 - 8s - loss: 7.8785 - cindex_score: 0.5073 - val_loss: 1.8718 - val_cindex_score: 0.4887
Epoch 2/100
Epoch 00002: val_loss improved from 1.87180 

KeyboardInterrupt: 

In [ ]:
with open(FLAGS.results_pickle[0], 'rb') as handle:
     results = pickle.load(handle)
#pd.DataFrame(results['b1c21098f5adaae3006535fec0f014b'], index=['b1c21098f5adaae3006535fec0f014b'])


In [5]:
df_results = pd.concat([pd.DataFrame(val, index=[i]) for i,val in results.items()], axis=0)
#best_rmse_result = df_results.cindex.idxmax()
#results[best_rmse_result]
#results['bf6405caa070526f50fa988e9cfc29b']
df_results

,FLAGS,cindex,f1,loss,n_repeats,params,rmse
beaf5b1662e9d26d052a96a2ec17433,NaN,0.588214,0,0.697925,1,NaN,0.835419
bb8685acc484adb82103aacc7c868e4,NaN,0.535844,0,0.641886,1,NaN,0.801178
b9711fe04a1d681bd79abb7cb9b1c93,NaN,0.543310,0,0.670153,1,NaN,0.818629
be84a46ff05014d7be837c403cc703b,NaN,0.503790,0,0.813888,1,NaN,0.902158


In [ ]:
gridmodel = load_model('checkpoints/davis_dtc_dta_bf6405caa070526f50fa988e9cfc29bfold0.h5')

predicted_labels = gridmodel.predict([np.array(XD_train), np.array(XT_train)])[:, 0]
loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)
print(loss, rperf2)
print(np.sum(predicted_labels>7), np.sum(Y_val>7))

In [ ]:
#gridmodel = load_model(model_name+'.h5')
data_path = '../data/'
XD_test, XT_test = get_DTC_train(data_path+'dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])


In [ ]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv(data_path+'submission_file.csv', index=None)